# RAM Concept Creep Factor <v0.1>

## Introduction:

This notebook aids in the calculation of the RAM Concept concrete creep factor. The RAM Concept creep factor defaults to a value of 3.35 based on recommendations in ACI 209.

### Assumptions and Limitations:
- Valid only for moist-cured slab elements with age at loading greater than 7 days.

In [1]:
import ipywidgets as widgets
from IPython.display import display
import json
from tkinter import Tk
from tkinter.filedialog import askopenfilename, asksaveasfilename
import pickle

In [2]:
class Creep_Factor(object):
    
    def __init__(self):
        
        # Calc Admin Data
        self.company = 'Company Name'
        self.proj_id = 'Project ID'
        self.date = 'Date'
        self.prep_by = 'ID of preparer'
        self.check_by = 'ID of checker'
        
        self.t0 = 14
        self.h = 8
        self.s = 3.5
        self.p = 40
        self.a = 1.5
        self.RH = 0.7
        
    def calc(self):
        
        # Calculate the concrete creep factor.
        self.gamma_ct0 = 1.25*self.t0**-0.118
        self.gamma_ch = 1.10 - 0.017*self.h
        self.gamma_cs = 0.82 + 0.067*self.s
        self.gamma_cpsi = 0.88 + 0.0024*self.p
        self.gamma_calpha = max(1, 0.46 + 0.09*self.a)
        self.gamma_cRH = 1.27 - 0.67*self.RH
        self.gamma_c = self.gamma_ct0*self.gamma_ch*self.gamma_cs*self.gamma_cpsi*self.gamma_calpha*self.gamma_cRH
        self.phi_u = 2.35*self.gamma_c
        self.k_e = self.phi_u + 1

# Create the calc object.
cf = Creep_Factor()

In [3]:
class Creep_Factor_Layout(object):
    
    def __init__(self, calc):
        
        self.cf = calc

        # Create calc admin widgets.
        self.company = widgets.Text(value=calc.company, description='Company:')
        self.proj_id = widgets.Text(value=calc.proj_id, description='Project ID:')
        self.date = widgets.Text(value=calc.date, description='Date:')
        self.prep_by = widgets.Text(value=calc.prep_by, description='Prepared By:')
        self.check_by = widgets.Text(value=calc.check_by, description='Checked By:')
        
        # Create calc input widgets.
        self.t0 = widgets.FloatText(value=calc.t0, description=r'$t_0$ &nbsp <i>[days]</i>', disabled=False)
        self.h = widgets.FloatText(value=calc.h, description=r'$h$ &nbsp <i>[in]</i>', disabled=False)
        self.s = widgets.FloatText(value=calc.s, description=r'$s$ &nbsp <i>[in]</i>', disabled=False)
        self.p = widgets.FloatText(value=calc.p, description=r'$\psi$ &nbsp <i>[%]</i>', disabled=False)
        self.a = widgets.FloatText(value=calc.a, description=r'$\alpha$ &nbsp <i>[%]</i>', disabled=False)
        self.RH = widgets.FloatText(value=calc.RH, description=r'$RH$ &nbsp <i>[u]</i>', disabled=False)
        
        # Create action widgets.
        self.update_info_btn = widgets.Button(description='Update Proj. Info.')
        self.update_info_btn.on_click(self.update_info)
        
        self.calc_btn = widgets.Button(description='Calc')
        self.calc_btn.on_click(self.calc)
        
#         self.save_file_btn = widgets.Button(description='Save')
#         self.save_file_btn.on_click(self.save_file)
        
        self.uploader = widgets.FileUpload()
        
        self.output = widgets.Output()
        with self.output:
            print('needs calc')
        
        # Create the dictionary for saving/loading calc.
        self.data = {
            'project info': {
                'company': calc.company,
                'project id': calc.proj_id,
                'date': calc.date,
                'prepared by': calc.prep_by,
                'checked by': calc.check_by},
            'input': {
                't0': calc.t0,
                'h': calc.h,
                's': calc.s,
                'p': calc.p,
                'a': calc.a,
                'RH': calc.RH},
            'output': {
                'gamma_ct0': 'needs calc',
                'gamma_ch': 'needs calc',
                'gamma_cs': 'needs calc',
                'gamma_cpsi': 'needs calc',
                'gamma_calpha': 'needs calc',
                'gamma_cRH': 'needs calc',
                'gamma_c': 'needs calc',
                'phi_u': 'needs calc',
                'k_e': 'needs calc'}}
        
    def update_info(self, b):
        
        # Update the calc.
        self.cf.company = self.company.value
        self.cf.proj_id = self.proj_id.value
        self.cf.date = self.date.value
        self.cf.prep_by = self.prep_by.value
        self.cf.check_by = self.check_by.value
        
        # Update the data.
        self.data['project info']['company'] = self.cf.company
        self.data['project info']['proj_id'] = self.cf.proj_id
        self.data['project info']['date'] = self.cf.date
        self.data['project info']['prep_by'] = self.cf.prep_by
        self.data['project info']['check_by'] = self.cf.check_by
        
    def calc(self, b):
        
        # Update the calc.
        self.cf.t0 = self.t0.value
        self.cf.h = self.h.value
        self.cf.s = self.s.value
        self.cf.p = self.p.value
        self.cf.a = self.a.value
        self.cf.RH = self.RH.value
        self.cf.calc()
        
        # Clear output.
        self.output.clear_output()

        # Save data.
        self.data['input']['t0'] = self.cf.t0
        self.data['input']['h'] = self.cf.h
        self.data['input']['s'] = self.cf.s
        self.data['input']['p'] = self.cf.p
        self.data['input']['a'] = self.cf.a
        self.data['input']['RH'] = self.cf.RH
        self.data['output']['gamma_ct0'] = self.cf.gamma_ct0
        self.data['output']['gamma_ch'] = self.cf.gamma_ch
        self.data['output']['gamma_cs'] = self.cf.gamma_cs
        self.data['output']['gamma_cpsi'] = self.cf.gamma_cpsi
        self.data['output']['gamma_calpha'] = self.cf.gamma_calpha
        self.data['output']['gamma_cRH'] = self.cf.gamma_cRH
        self.data['output']['gamma_c'] = self.cf.gamma_c
        self.data['output']['phi_u'] = self.cf.phi_u
        self.data['output']['k_e'] = self.cf.k_e
        
        # Create output.
        with self.output:
            display(widgets.HTMLMath(
                rf"""
                <h3> User Input </h3>
                $t_0 = \boldsymbol{{ {self.cf.t0} }}$ days &nbsp; <i>(age at loading)</i> <br>
                $h = \boldsymbol{{ {self.cf.h} }}$ in &nbsp; <i>(slab thickness)</i> <br>
                $s = \boldsymbol{{ {self.cf.s} }}$ in &nbsp; <i>(slump)</i> <br>
                $\psi = \boldsymbol{{ {self.cf.p} }}$ % &nbsp; <i>(fine aggregate to toal aggregate)</i> <br>
                $\alpha = \boldsymbol{{ {self.cf.a} }}$ % &nbsp; <i>(air content)</i> <br>
                $RH = \boldsymbol{{ {self.cf.RH} }}$ &nbsp; <i>(relative humidity)</i> <br>

                <h3> Calculate Creep Factor </h3>
                $\gamma_{{c,t0}} = 1.25t_0^{{-0.118}} = \boldsymbol{{ {self.cf.gamma_ct0:.2f} }}$ &nbsp; <i>(age at loading factor)</i> <br>
                $\gamma_{{c,h}} = 1.10 - 0.017h = \boldsymbol{{ {self.cf.gamma_ch:.2f} }}$ &nbsp; <i>(thickness factor)</i> <br>
                $\gamma_{{c,s}} = 0.82 + 0.067s = \boldsymbol{{ {self.cf.gamma_cs:.2f} }}$ &nbsp; <i>(slump factor)</i> <br>
                $\gamma_{{c,\psi}} = 0.88 + 0.0024\psi = \boldsymbol{{ {self.cf.gamma_cpsi:.2f} }}$ &nbsp; <i>(fine aggregate to total aggregate factor)</i> <br>
                $\gamma_{{c,\alpha}} = max(1, 0.46 + 0.09\alpha) = \boldsymbol{{ {self.cf.gamma_calpha:.2f} }}$ &nbsp; <i>(air content factor)</i> <br>
                $\gamma_{{c,RH}} = 1.27 - 0.67RH = \boldsymbol{{ {self.cf.gamma_cRH:.2f} }}$ &nbsp; <i>(relative humidity factor)</i> <br>
                $\gamma_c = \gamma_{{c,t0}}\gamma_{{c,RH}}\gamma_{{c,d}}\gamma_{{c,s}}\gamma_{{c,\psi}}\gamma_{{c,\phi}}\gamma_{{c,\alpha}} = \boldsymbol{{ {self.cf.gamma_c:.2f} }}$ &nbsp; <i>(correction factor)</i>  <br>
                    <blockquote>
                    $\phi_u = 2.35\gamma_c = \boldsymbol{{ {self.cf.phi_u:.2f} }}$ &nbsp; <i>(creep coefficient)</i> <br>
                        <blockquote>
                        $k_{{e}} = \phi_{{u}} + 1 = \boldsymbol{{ {self.cf.k_e:.2f} }}$ &nbsp; <i>(Concept creep factor)</i>
                        </blockquote>
                    </blockquote>
                """))

#     def save_file(self, b):

#         # Create Tk root
#         root = Tk()
#         # Hide the main window
#         root.withdraw()
#         # Get the file path with a file browser.
#         file_path = asksaveasfilename(defaultextension='.pkl')
#         # Launch the GUI.
#         #%gui tk
        
#         if file_path != "":
#             # Pickle the object.
#             pickle_out = open(file_path, "wb")
#             pickle.dump(self.model, pickle_out)
#             pickle_out.close()
        
#         # Save file.
#         with open('data.txt', 'w') as outfile:
#             json.dump(self.data, outfile)
    

In [4]:
# Create the calc layout.
cfl = Creep_Factor_Layout(cf)

## Save/Load:

In [5]:
# display(cfl.save_file_btn)
display(cfl.uploader)

FileUpload(value={}, description='Upload')

## Project Information:

In [6]:
display(cfl.company, cfl.proj_id, cfl.date, cfl.prep_by, cfl.check_by)
display(cfl.update_info_btn)

Text(value='Company Name', description='Company:')

Text(value='Project ID', description='Project ID:')

Text(value='Date', description='Date:')

Text(value='ID of preparer', description='Prepared By:')

Text(value='ID of checker', description='Checked By:')

Button(description='Update Proj. Info.', style=ButtonStyle())

## Inputs:
- $t_0$ [days] = age of slab at application of loading. E.g. $t_0 = 14$ for 14 days.
- $h$ [in] = slab depth.
- $s$ [in] = concrete mix slump.
- $psi$ [%] = $\psi$, ratio of fine aggregate to total aggregate (typically about 40%, i.e. $\psi = 40$).
- $\alpha$ [%] = $\alpha$, air content. E.g. $\alpha = 1.5$ for 1.5% air content.
- $RH$ = average relative humidity. E.g. $RH = 0.7$ for 70%.

Expected units are given with square brackets, '[  ]'.

In [7]:
display(cfl.t0, cfl.h, cfl.s, cfl.p, cfl.a, cfl.RH)
display(cfl.calc_btn)

FloatText(value=14.0, description='$t_0$ &nbsp <i>[days]</i>')

FloatText(value=8.0, description='$h$ &nbsp <i>[in]</i>')

FloatText(value=3.5, description='$s$ &nbsp <i>[in]</i>')

FloatText(value=40.0, description='$\\psi$ &nbsp <i>[%]</i>')

FloatText(value=1.5, description='$\\alpha$ &nbsp <i>[%]</i>')

FloatText(value=0.7, description='$RH$ &nbsp <i>[u]</i>')

Button(description='Calc', style=ButtonStyle())

## Results:

In [8]:
accordion = widgets.Accordion(children=[cfl.output])
accordion.set_title(0, 'Results')

display(accordion)

Accordion(children=(Output(),), _titles={'0': 'Results'})

## Discussion:
As noted in the RAM Concept manual, the creep coefficient ($\phi_u$) is commonly defined as follows. 
> $\phi_u = \frac{\epsilon_c}{\epsilon_i}$, where
>> $\epsilon_c =$ creep strain, and <br>
>> $\epsilon_i = \epsilon_e =$ initial (elastic) strain

> $\epsilon_f = \epsilon_i + \epsilon_c$, where
>> $\epsilon_f =$ total strain

RAM Concept uses the factor $k_e$ to symbolize the "concrete design creep factor" and defined it as follows.
> $k_e = \frac{\epsilon_f}{\epsilon_e}$, where $\epsilon_e = \epsilon_i$

Consequently, Concept uses 
> $k_e = \frac{\epsilon_i + \epsilon_c}{\epsilon_i} = 1 + \frac{\epsilon_c}{\epsilon_i} = 1 + \phi_u$

Since 2.35 is suggested as average creep coefficient, $\phi_u = 2.35$, Concept defaults to $k_e = 3.35$.

Average relative humidity for different regions in the U.S. may be found the PCI Design Handbook.

### References:

1. ACI 209.2R-08 - Guide for Modeling and Calculating Shrinkage and Creep in Hardened Concrete, ACI, 2008.
2. RAM Concept Manual, 2020.
3. Prestressed Concrete Analysis and Design - Fundamentals (3rd Ed.), Antoine E. Naaman, 2012.

### Change Log:
- Created: Dec. 21, 2020 by SJ
- Released:

### Comments:
- Where can each of the inputs be referenced? 
- How much difference does each factor make?
- Can we get a chart of creep factors for common situations and for reference (controls)?
- Location for user to add notes or description, maybe both.
- Put Discussion in an accordian so that it doesn't need to print.
- Need to figure out the save/load issue. Binder doesn't allow input/output of local files. BinderHub or Heroku might be options. Could also create a JupyterHub and run Voila with executables or run the full notebook. This last option does not require (benefit from) GitHub. On the other hand, the free Binder or Heroku is limited in RAM and use-time, and probably other ways.